In [1]:
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import optuna
from collections import defaultdict

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso

from src.utils import get_kfold_data, convert_non_numeric_to_numeric, calculate_r2_score, calculate_metrics
from src.normalisation import Normaliser
from src.constants import *


c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv(DATA_PATH)

In [3]:
data.head()

,outcome,carat,cut,color,clarity,depth,table,price,x,y,...,a6,a7,a8,a9,a10,b6,b7,b8,b9,b10
0,-26.701232,1.14,Ideal,G,VS1,62.3,56.0,7948,6.73,6.70,...,0.168836,-0.273758,1.107832,1.247795,0.482344,0.489511,-0.321138,0.573382,0.446871,-1.990581
1,6.548093,0.38,Premium,H,VS2,60.5,59.0,898,4.69,4.66,...,-0.256549,0.315373,-0.030326,-0.114335,-1.059588,-1.761360,-1.343951,-1.002550,-0.225030,-0.446653
2,6.612562,0.50,Very Good,E,SI1,60.7,58.0,1351,5.09,5.13,...,-1.193327,-0.657307,-0.591726,-0.446856,-0.765286,-0.816544,-1.397794,-0.477130,0.810509,1.725131
3,-5.073562,0.70,Premium,D,SI1,61.2,58.0,2512,5.74,5.70,...,-1.740788,-1.778860,-0.825070,0.444932,1.173109,0.453606,-0.263440,0.246210,-0.850503,-0.412950
4,-14.436557,0.83,Ideal,G,SI2,62.4,54.0,2751,6.01,6.08,...,-0.859322,1.409268,0.861992,1.109063,-1.436722,-1.461618,0.081787,0.258087,0.851146,2.204813


Inspecting columns

In [4]:
# Find columns
all_columns = data.columns.tolist()
print(all_columns)

numeric_columns = data.select_dtypes(include=["number"]).columns.tolist()
numeric_columns.remove("outcome") # Remove the target column
print(numeric_columns)

non_numeric_columns = data.select_dtypes(exclude=["number"]).columns.tolist()
print(non_numeric_columns)

['outcome', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['cut', 'color', 'clarity']


In [5]:
for non_numeric_column in non_numeric_columns:
    print(data[non_numeric_column].value_counts())

cut
Ideal        4040
Premium      2439
Very Good    2296
Good          925
Fair          300
Name: count, dtype: int64
color
G    2120
E    1873
F    1746
H    1506
D    1246
I     983
J     526
Name: count, dtype: int64
clarity
SI1     2408
VS2     2256
SI2     1743
VS1     1503
VVS2     951
VVS1     675
IF       318
I1       146
Name: count, dtype: int64


Converting non-numeric features to numerical features

In [6]:
data = convert_non_numeric_to_numeric(data=data)
print(data)

['G', 'E', 'F', 'H', 'D', 'I', 'J']
        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9996 -12.246698   1.01    4        5   69.5   55.0   4853  6.00  5.94  4.15   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...      

Normalise data using each columns respective mean and std.

In [7]:
print(data)

        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9996 -12.246698   1.01    4        5   69.5   55.0   4853  6.00  5.94  4.15   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...        b8        b9       b10  colour_G  

In [8]:
normaliser = Normaliser()
for column in numeric_columns:
    print(data[column])
    data[column] = normaliser.standardise(data[column])
    print("after", data[column])
    break

0       1.14
1       0.38
2       0.50
3       0.70
4       0.83
        ... 
9995    0.33
9996    1.01
9997    0.52
9998    0.31
9999    0.37
Name: carat, Length: 10000, dtype: float64
after 0       0.723643
1      -0.886369
2      -0.632156
3      -0.208469
4       0.066928
          ...   
9995   -0.992290
9996    0.448246
9997   -0.589788
9998   -1.034659
9999   -0.907553
Name: carat, Length: 10000, dtype: float64


In [9]:
print(data)

        outcome     carat  cut  clarity  depth  table  price     x     y  \
0    -26.701232  0.723643    0        3   62.3   56.0   7948  6.73  6.70   
1      6.548093 -0.886369    1        4   60.5   59.0    898  4.69  4.66   
2      6.612562 -0.632156    2        5   60.7   58.0   1351  5.09  5.13   
3     -5.073562 -0.208469    1        5   61.2   58.0   2512  5.74  5.70   
4    -14.436557  0.066928    0        6   62.4   54.0   2751  6.01  6.08   
...         ...       ...  ...      ...    ...    ...    ...   ...   ...   
9995  10.718277 -0.992290    0        3   62.6   57.0   1002  4.42  4.40   
9996 -12.246698  0.448246    4        5   69.5   55.0   4853  6.00  5.94   
9997  11.122516 -0.589788    2        6   57.9   61.0   1273  5.28  5.33   
9998 -24.730782 -1.034659    0        0   62.0   54.0    801  4.35  4.39   
9999   8.735755 -0.907553    2        5   59.9   59.0    649  4.68  4.70   

         z  ...        b8        b9       b10  colour_G  colour_E  colour_F  \
0     4.

In [10]:
kfold_data = get_kfold_data(data=data, k=NUM_FOLDS, reproducibility_seed=REPRODUCIBILITY_SEED)

Fold: 0/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 1/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 2/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 3/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 4/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%



Define models and hyperparameter tuning objectives for each model

In [11]:
models = {
        "linear_regression": LinearRegression,
        "lasso": Lasso,
        "ridge": Ridge,
        # "xgb": xgb.XGBRegressor,
        # "random_forest": RandomForestRegressor,
        # "gradient_boosting": GradientBoostingRegressor,
        # "ada_boost": AdaBoostRegressor,
        # "lgbm": lgb.LGBMRegressor
        }

def objective(model_type, trial, x_train, y_train, x_val, y_val):
    if model_type == LinearRegression:
        parameters = {
            "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        }
    elif model_type == Lasso:
        parameters = {
            "alpha": trial.suggest_float("alpha", 1e-3, 0.1, log=True),
            "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
            "selection": trial.suggest_categorical("selection", ["cyclic", "random"]),
            "warm_start": trial.suggest_categorical("warm_start", [True, False]),
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == Ridge:
        parameters = {
            "alpha": trial.suggest_float("alpha", 1e-3, 0.1, log=True),
            "solver": trial.suggest_categorical("solver", ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"]),
            "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
            "positive": False,
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == xgb.XGBRegressor:
        parameters = {
            "objective": "reg:squarederror",
            "eval_metric": "rmse",
            "n_estimators": 100,
            "eta": trial.suggest_float("eta", 1e-2, 0.2, log=True),
            "gamma": trial.suggest_float("gamma", 1e-8, 10, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 6),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "seed": REPRODUCIBILITY_SEED
        }
    elif model_type == RandomForestRegressor:
        parameters = {
            "n_estimators": 100,
            "criterion": trial.suggest_categorical("criterion", ["absolute_error", "squared_error"]),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "bootstrap": True,
            "oob_score": False,
            "n_jobs": -1,
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == GradientBoostingRegressor:
        parameters = {
            "n_estimators": 100,
            "loss": trial.suggest_categorical("loss", ["absolute_error", "squared_error", "huber", "quantile"]),
            "criterion": trial.suggest_categorical("criterion", ["friedman_mse", "squared_error"]),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
            "subsample": trial.suggest_float("subsample", 0.05, 1.0),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
            "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 2, 2**10),
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == AdaBoostRegressor:
        parameters = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 100),
            "loss": trial.suggest_categorical("loss", ["linear", "square", "exponential"]),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
            "random_state": REPRODUCIBILITY_SEED
        }
    elif model_type == lgb.LGBMRegressor:
        parameters = {
                    "objective": "regression",
                    "metric": "rmse",
                    "n_estimators": 100,
                    "verbosity": -1,
                    "bagging_freq": 1,
                    "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
                    "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
                    "subsample": trial.suggest_float("subsample", 0.05, 1.0),
                    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
                    "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
                    "seed": REPRODUCIBILITY_SEED
        }

    
    model = model_type(**parameters) # Create the model
    model.fit(x_train, y_train)
    predictions = model.predict(x_val)
    metrics = calculate_metrics(targets=y_val, preds=predictions)
    rmse = metrics["rmse"]
    return rmse

In [12]:
# Train + Validate models
metrics = ["mae", "mse", "rmse", "pcc", "spearman_r", "r2_score"]
model_scores = {model_name: defaultdict(list) for model_name in models.keys()}

for fold in range(NUM_FOLDS):
    fold_data = kfold_data[fold]
     
    # Extract data
    train_data = fold_data["train"]
    val_data = fold_data["val"]
    test_data = kfold_data[fold]["test"]

    train_y = train_data["outcome"]
    val_y = val_data["outcome"]
    test_y = test_data["outcome"]

    train_x = train_data.drop(columns=["outcome"])
    val_x = val_data.drop(columns=["outcome"])
    test_x = test_data.drop(columns=["outcome"])

    # print(f"Fold {fold+1}/{NUM_FOLDS}")
    # print(f"Train data shape: {train_x.shape} | Train target shape: {train_y.shape}")
    # print(f"Val data shape: {val_x.shape} | Val target shape: {val_y.shape}")
    # print(f"Test data shape: {test_x.shape} | Test target shape: {test_y.shape}")

    # Train model
    for model_name, model in models.items():
        study = optuna.create_study(direction="minimize")
        study.optimize(lambda trial: objective(trial=trial, 
                                               model_type=model, 
                                               x_train=train_x, 
                                               y_train=train_y, 
                                               x_val=val_x, 
                                               y_val=val_y
                                               ), n_trials=30)
        
        # Train model with best hyperparameters
        best_fold_params = study.best_params
        model = model(**best_fold_params)
        model.fit(train_x, train_y)
        preds = model.predict(val_x)

        metrics = calculate_metrics(targets=val_y, preds=preds)
        mae = metrics["mae"]
        mse = metrics["mse"]
        rmse = metrics["rmse"]
        pcc = metrics["pcc"]
        spearman_r = metrics["spearman_r"]
        r2_score = metrics["r2_score"]

        for metric in metrics:
            model_scores[model_name][metric].append(metrics[metric])

        print(f"Fold: {fold+1}/{NUM_FOLDS}")
        print(f"Model name: {model_name}")
        print(f"MAE: {mae}")
        print(f"MSE: {mse}")
        print(f"RMSE: {rmse}")
        print(f"PCC: {pcc}")
        print(f"Spearman R: {spearman_r}")
        print(f"R2 Score: {r2_score}")
        print()


[I 2025-02-16 15:54:25,363] A new study created in memory with name: no-name-1e73f171-c67b-4346-a959-15aae628298f
[I 2025-02-16 15:54:25,371] Trial 0 finished with value: 10.913995409241268 and parameters: {'fit_intercept': False}. Best is trial 0 with value: 10.913995409241268.
[I 2025-02-16 15:54:25,377] Trial 1 finished with value: 10.913995409241268 and parameters: {'fit_intercept': False}. Best is trial 0 with value: 10.913995409241268.
[I 2025-02-16 15:54:25,384] Trial 2 finished with value: 10.913995409241357 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 10.913995409241268.
[I 2025-02-16 15:54:25,391] Trial 3 finished with value: 10.913995409241357 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 10.913995409241268.
[I 2025-02-16 15:54:25,399] Trial 4 finished with value: 10.913995409241357 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 10.913995409241268.
[I 2025-02-16 15:54:25,406] Trial 5 finished with value: 10

Fold: 1/5
Model name: linear_regression
MAE: 8.775370267692598
MSE: 119.11529579293946
RMSE: 10.913995409241268
PCC: 0.5463216691195159
Spearman R: 0.5697531327160675
R2 Score: 0.2980010974654729



c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.875e+05, tolerance: 1.174e+02
  model = cd_fast.enet_coordinate_descent(
[I 2025-02-16 15:54:25,836] Trial 3 finished with value: 11.273323143647062 and parameters: {'alpha': 0.010615217101021172, 'fit_intercept': False, 'selection': 'random', 'warm_start': False}. Best is trial 0 with value: 10.926487825178297.
c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.866e+05, tolerance: 1.174e+02
  model = cd_fast.enet_coordinate_descen

Fold: 1/5
Model name: lasso
MAE: 8.776322295049523
MSE: 119.10902688904005
RMSE: 10.913708209817598
PCC: 0.546377000548236
Spearman R: 0.5703277940733571
R2 Score: 0.29803804287729585



c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-02-16 15:54:27,764] Trial 0 finished with value: 12.926647637874595 and parameters: {'alpha': 0.004464436425327533, 'solver': 'sag', 'fit_intercept': False}. Best is trial 0 with value: 12.926647637874595.
[I 2025-02-16 15:54:27,768] Trial 1 finished with value: 12.36509822201116 and parameters: {'alpha': 0.00407719208742347, 'solver': 'lsqr', 'fit_intercept': False}. Best is trial 1 with value: 12.36509822201116.
[I 2025-02-16 15:54:27,772] Trial 2 finished with value: 10.914505138047922 and parameters: {'alpha': 0.0017320276276025978, 'solver': 'cholesky', 'fit_intercept': False}. Best is trial 2 with value: 10.914505138047922.
[I 2025-02-16 15:54:27,781] Trial 3 finished with value: 10.934714164874391 and parameters: {'alpha': 0.03239891726246961, 'solver': 'svd', 'fit_in

...

Fold: 5/5
Model name: linear_regression
MAE: 8.596058566883377
MSE: 118.93968797570939
RMSE: 10.905947367180413
PCC: 0.5235762981647845
Spearman R: 0.5555014425786886
R2 Score: 0.2721068246878642



c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.118e+05, tolerance: 1.194e+02
  model = cd_fast.enet_coordinate_descent(
[I 2025-02-16 15:54:52,667] Trial 4 finished with value: 11.174906823330469 and parameters: {'alpha': 0.06705168438753978, 'fit_intercept': False, 'selection': 'random', 'warm_start': True}. Best is trial 0 with value: 10.786997748895105.
[I 2025-02-16 15:54:52,680] Trial 5 finished with value: 10.78163631608544 and parameters: {'alpha': 0.03392813672580918, 'fit_intercept': True, 'selection': 'random', 'warm_start': True}. Best is trial 5 with value: 10.78163631608544.
c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Obj

Fold: 5/5
Model name: lasso
MAE: 8.548655948256824
MSE: 116.16871584756348
RMSE: 10.778159204964616
PCC: 0.5377233628482976
Spearman R: 0.5656504465431432
R2 Score: 0.2890647613983546



c:\Users\wlyle\Desktop\DataScienceDataChallenge\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-02-16 15:54:54,204] Trial 2 finished with value: 12.701612236509364 and parameters: {'alpha': 0.0072948414253001985, 'solver': 'saga', 'fit_intercept': True}. Best is trial 0 with value: 10.902418370052766.
[I 2025-02-16 15:54:54,208] Trial 3 finished with value: 10.82441652537743 and parameters: {'alpha': 0.022002739794985605, 'solver': 'auto', 'fit_intercept': False}. Best is trial 3 with value: 10.82441652537743.
[I 2025-02-16 15:54:54,212] Trial 4 finished with value: 12.08914074400373 and parameters: {'alpha': 0.0011979886979986486, 'solver': 'lsqr', 'fit_intercept': False}. Best is trial 3 with value: 10.82441652537743.
[I 2025-02-16 15:54:54,216] Trial 5 finished with value: 11.760172561865351 and parameters: {'alpha': 0.0027931935015299923, 'solver': 'lsqr', 'fit_int

Fold: 5/5
Model name: ridge
MAE: 8.581201472894989
MSE: 116.64402089752144
RMSE: 10.80018615105876
PCC: 0.5355512434348788
Spearman R: 0.556710000667969
R2 Score: 0.2861559652854334



Compute average scores and rank models by R2 score

In [13]:
for model_name, model_metrics in model_scores.items():
    for metric, scores in model_metrics.items():
        model_scores[model_name][metric] = sum(scores) / len(scores)
    model_scores[model_name] = dict(model_scores[model_name])

model_scores = dict(sorted(model_scores.items(), key=lambda x: x[1]["r2_score"], reverse=True))

In [14]:
# Print results
for i, (model_name, model_metrics) in enumerate(model_scores.items()):
    print(f"No.{i+1} Model: {model_name}")
    for metric, score in model_metrics.items():
        print(f"{metric}: {score}")
    print()

No.1 Model: lasso
mae: 8.55142996065934
mse: 115.22621273770429
rmse: 10.733397920563233
pcc: 0.5432106763598974
spearman_r: 0.57210927387081
r2_score: 0.2943863424292353

No.2 Model: ridge
mae: 8.569002385106042
mse: 115.43577237504641
rmse: 10.743154074464211
pcc: 0.5421809362717445
spearman_r: 0.567962929477707
r2_score: 0.29309029601575254

No.3 Model: linear_regression
mae: 8.57109532257773
mse: 115.90552181588734
rmse: 10.764799800447392
pcc: 0.5397327873066068
spearman_r: 0.5680140097710977
r2_score: 0.29021532307368103

